<a href="https://colab.research.google.com/github/bojunchen3/Project/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install larq
!pip uninstall sklearn
!pip install pretty_confusion_matrix
%mkdir dataset
%cd /content/drive/MyDrive/project
!unrar x /content/drive/MyDrive/project/dataset.rar -d /content/dataset

  Using cached pretty_confusion_matrix-0.1.1-py3-none-any.whl (9.6 kB)
  Using cached black-21.12b0-py3-none-any.whl (156 kB)
  Using cached flake8-3.9.2-py2.py3-none-any.whl (73 kB)
  Using cached isort-5.12.0-py3-none-any.whl (91 kB)
  Using cached pre_commit-2.21.0-py2.py3-none-any.whl (201 kB)
  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
  Using cached sklearn-0.0.post7.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 40.0 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post7-py3-none-any.whl size=2951 sha256=782f588aea3fcbea614f731684aa6f140b6fb41b5c23aac4b4dabc3ece8d8507

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import Softmax, Dropout, Flatten, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from larq.layers import QuantConv2D
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
import larq as lq
import os
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
# 導入資料集
train = ImageDataGenerator(rescale=1/255)
val = ImageDataGenerator(rescale=1/255)
train_dataset = train.flow_from_directory('/content/dataset/train/',
                      target_size=(46,46),
                      batch_size=128,
                      color_mode='grayscale',
                      class_mode='categorical')
val_dataset = val.flow_from_directory('/content/dataset/validation/',
                    target_size=(46,46),
                    batch_size=128,
                    color_mode='grayscale',
                    class_mode='categorical')
kwargs = dict(kernel_quantizer="ste_sign",
        kernel_constraint="weight_clip")
# 定義模型架構
model = Sequential()
model.add(QuantConv2D(filters=8, kernel_size=(3, 3), input_shape=(46, 46, 1), **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=16, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=32, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=32, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=32, kernel_size=(1, 1), input_quantizer="ste_sign", **kwargs))
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=10, kernel_size=(1, 1), input_quantizer="ste_sign", activation='softmax', **kwargs))
model.add(Flatten())

adam = Adam(learning_rate=0.005)
model.compile(optimizer=adam,
       loss='categorical_crossentropy',
       metrics=['accuracy'])
# 定義學習率調整函數
def lr_scheduler(epoch, learning_rate):
    if epoch > 0 and epoch % 20 ==0:
        return learning_rate * 0.5
    else:
        return learning_rate

# 創建學習率調度器
lr_callback = LearningRateScheduler(lr_scheduler)
# 早停機制
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0,
                  patience=10, mode='auto')
# 斷點續訓
checkpoint_callback = ModelCheckpoint('checkpoint.ckpt', monitor='val_accuracy',
                    save_weights_only=True, verbose=1,
                    save_best_only=True, mode='auto')
'''
if os.path.exists('checkpoint'):
    model.load_weights('checkpoint.ckpt')
    print("Cotinue with checkpoint")
else:
    print("No checkpoint")
'''
print('Training ---------------------------------')
history = model.fit(train_dataset,
           epochs=100,
           batch_size=128,
           validation_data=val_dataset,
           callbacks=[lr_callback,
                earlystop_callback,
                checkpoint_callback])
model.save('model.h5')

# 繪製圖表
plt.figure()
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']
plt.subplot(2, 1, 1)
plt.title('model loss')
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend(['train_loss',
      'valid_loss'],
      loc='upper right')
plt.subplot(2, 1, 2)
plt.title('model accuracy')
plt.plot(acc, label='acc')
plt.plot(val_acc, label='val_acc')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend(['train_accuracy',
      'valid_accuracy'],
      loc='upper right')
plt.tight_layout()
plt.savefig('./result.png')


Found 5000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Training ---------------------------------
Epoch 1/100
40/40 [==============================] - ETA: 0s - loss: 3.2624 - accuracy: 0.4674
Epoch 1: val_accuracy improved from -inf to 0.55100, saving model to checkpoint.ckpt
40/40 [==============================] - 25s 196ms/step - loss: 3.2624 - accuracy: 0.4674 - val_loss: 2.4744 - val_accuracy: 0.5510 - lr: 0.0050
Epoch 2/100
40/40 [==============================] - ETA: 0s - loss: 1.1231 - accuracy: 0.7338
Epoch 2: val_accuracy improved from 0.55100 to 0.60400, saving model to checkpoint.ckpt
40/40 [==============================] - 6s 142ms/step - loss: 1.1231 - accuracy: 0.7338 - val_loss: 2.3379 - val_accuracy: 0.6040 - lr: 0.0050
Epoch 3/100
40/40 [==============================] - ETA: 0s - loss: 0.7236 - accuracy: 0.8298
Epoch 3: val_accuracy improved from 0.60400 to 0.69600, saving model to checkpoint.ckpt
40/40 [=============================

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from pretty_confusion_matrix import pp_matrix_from_data
from keras.models import load_model
import numpy as np
import cv2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

test = ImageDataGenerator(rescale=1/255)
test_dataset = test.flow_from_directory('/content/dataset/test/',
                     target_size=(46,46),
                     batch_size=256,
                     color_mode='grayscale',
                     class_mode='categorical',
                     shuffle=False)

model = load_model('model.h5')
print('\nTesting ------------')
loss, accuracy = model.evaluate(test_dataset)
print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

j=1
k=0
for i in 'down','left','ok','paper','right','rock','scissors','stone','thumb','up':
    x = cv2.imread('/content/dataset/test/{}/{}.jpg'.format(i,j))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
    x = x/255
    x = cv2.resize(x, (46, 46))
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    #print(pred)
    print(np.argmax(pred)+1)
    if np.argmax(pred)+1 == j:
      print('correct')
      k+=1
    else:
      print('false')
    j+=1
print(k)


# 預測測試資料
Y_pred = model.predict(test_dataset)
# 將預測結果轉成類別編號
y_pred = np.argmax(Y_pred, axis=1)
# 取得真實類別編號
y_true = test_dataset.classes
# 繪製混淆矩陣
class_name = ['down',
        'left',
        'ok',
        'paper',
        'right',
        'rock',
        'scissors',
        'stone',
        'thumb',
        'up']
cmap = 'terrain'
pp_matrix_from_data(y_true, y_pred, cmap=cmap, columns=class_name)
plt.savefig('./confusion.png')


Found 1000 images belonging to 10 classes.

Testing ------------
4/4 [==============================] - 2s 321ms/step - loss: 0.4652 - accuracy: 0.9180

test loss:  0.4651755392551422

test accuracy:  0.9179999828338623
1/1 [==============================] - 1s 548ms/step
1
correct
1/1 [==============================] - 0s 30ms/step
2
correct
1/1 [==============================] - 0s 36ms/step
3
correct
1/1 [==============================] - 0s 33ms/step
7
false
1/1 [==============================] - 0s 40ms/step
5
correct
1/1 [==============================] - 0s 45ms/step
6
correct
1/1 [==============================] - 0s 33ms/step
7
correct
1/1 [==============================] - 0s 77ms/step
8
correct
1/1 [==============================] - 0s 31ms/step
9
correct
1/1 [==============================] - 0s 32ms/step
10
correct
9
4/4 [==============================] - 4s 832ms/step


In [ ]:
with lq.context.quantized_scope(True):
   model.set_weights(model.get_weights())

for layer in model.layers:
  if 'conv' in layer.name:
    print(layer.name)
    weight, bias = model.get_layer(layer.name).get_weights()
    print(np.shape(weight))
    f = open('./weight/W_'+layer.name+'.bin', 'w')
    for i in range(np.shape(weight)[0]):
      for j in range(np.shape(weight)[1]):
        for k in range(np.shape(weight)[2]):
          for l in range(np.shape(weight)[3]):
            f.write(str(weight[i][j][k][l])+"\n")
    f.close()
    f = open('./weight/b_'+layer.name+'.bin', 'w')
    for i in range(np.shape(bias)[0]):
      f.write(str(bias[i])+"\n")
    f.close()

for layer in model.layers:
  if 'batch_normalization' in layer.name:
    print(layer.name)
    a = model.get_layer(layer.name).get_weights()
    weight = a[0]/pow(a[3]+0.001, 0.5)
    bias = -(a[0]*a[2])/pow(a[3]+0.001, 0.5) + a[1]
    print(np.shape(weight))
    f = open('./weight/W_'+layer.name+'.bin', 'w')
    for i in range(np.shape(weight)[0]):
      f.write(str(weight[i])+"\n")
    f.close()
    f = open('./weight/b_'+layer.name+'.bin', 'w')
    for i in range(np.shape(bias)[0]):
      f.write(str(bias[i])+"\n")
    f.close()

quant_conv2d
(3, 3, 1, 8)
quant_conv2d_1
(3, 3, 8, 16)
quant_conv2d_2
(3, 3, 16, 32)
quant_conv2d_3
(3, 3, 32, 32)
quant_conv2d_4
(1, 1, 32, 32)
quant_conv2d_5
(1, 1, 32, 10)
batch_normalization
(8,)
batch_normalization_1
(16,)
batch_normalization_2
(32,)
batch_normalization_3
(32,)
batch_normalization_4
(32,)


In [ ]:
for layer in model.layers:
  if 'conv' in layer.name:
    weight, bias = model.get_layer(layer.name).get_weights()
    print(layer.name, weight.shape, bias.shape)

quant_conv2d (3, 3, 1, 8) (8,)
quant_conv2d_1 (3, 3, 8, 16) (16,)
quant_conv2d_2 (3, 3, 16, 32) (32,)
quant_conv2d_3 (3, 3, 32, 32) (32,)
quant_conv2d_4 (1, 1, 32, 32) (32,)
quant_conv2d_5 (1, 1, 32, 10) (10,)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quant_conv2d (QuantConv2D)  (None, 44, 44, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 8)        0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 22, 22, 8)        32        
 ormalization)                                                   
                                                                 
 quant_conv2d_1 (QuantConv2D  (None, 20, 20, 16)       1168      
 )                                                               
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 16)       0         
 2D)                                                    

In [ ]:
import cv2
import larq as lq
import numpy as np
from keras.models import Model
from keras.models import load_model
model = load_model('model.h5')
x = cv2.imread('/content/dataset/test/down/1.jpg')
x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
x = x/255
x = cv2.resize(x, (46, 46))
x = np.expand_dims(x, axis=0)
conv_model = Model(inputs=model.input, outputs=model.get_layer('quant_conv2d').output)
conv_output = conv_model.predict(x)
print(conv_output)


1/1 [==============================] - 0s 74ms/step
[[[[-0.6325317  -1.8141849  -0.60731727 ...  1.8651901   0.55306983
    -3.101766  ]
   [-0.56538695 -1.8561027  -0.6433749  ...  1.8820368   0.63792825
    -3.111837  ]
   [-0.6739619  -1.8770949  -0.7071781  ...  1.8962716   0.5994188
    -3.1098557 ]
   ...
   [-0.6014499  -1.9564679  -0.6420423  ...  1.9416889   0.68603057
    -3.2526336 ]
   [-0.6712451  -1.9979703  -0.7081382  ...  2.0044334   0.60271394
    -3.3210683 ]
   [-0.6593119  -1.9569998  -0.6733205  ...  1.9761432   0.63253146
    -3.3265448 ]]

  [[-0.58163476 -1.8401774  -0.6189652  ...  1.8576417   0.6246532
    -3.1390562 ]
   [-0.58848256 -1.8707771  -0.6186001  ...  1.8646604   0.58903474
    -3.110773  ]
   [-0.5907047  -1.884927   -0.61069196 ...  1.8609517   0.5795107
    -3.1717892 ]
   ...
   [-0.6518948  -1.9749544  -0.6760113  ...  1.9661912   0.64413685
    -3.297463  ]
   [-0.64877015 -1.9419286  -0.6651621  ...  1.9919108   0.6116672
    -3.325818  ]
 

In [ ]:
pool_model = Model(inputs=model.input, outputs=model.get_layer('max_pooling2d').output)
pool_output = pool_model.predict(x)
print(pool_output)

1/1 [==============================] - 0s 72ms/step
[[[[-0.56538695 -1.8141849  -0.60731727 ...  1.8820368   0.63792825
    -3.101766  ]
   [-0.5907047  -1.8165774  -0.61069196 ...  1.8962716   0.65577745
    -3.1098557 ]
   [-0.57851535 -1.8205786  -0.5938122  ...  1.8995631   0.648266
    -3.0975518 ]
   ...
   [-0.62914747 -1.9401385  -0.65625894 ...  2.0027602   0.6657796
    -3.2915545 ]
   [-0.6014499  -1.9006004  -0.6420423  ...  2.007493    0.68603057
    -3.2526336 ]
   [-0.6301169  -1.9358406  -0.65948176 ...  2.0044334   0.6648214
    -3.2792249 ]]

  [[-0.59832543 -1.7866449  -0.62989026 ...  1.9147402   0.6726906
    -3.0910878 ]
   [-0.5495005  -1.8351862  -0.6347794  ...  1.9186689   0.6742232
    -3.0929317 ]
   [-0.58590096 -1.8316262  -0.6132978  ...  1.9231056   0.654432
    -3.1341562 ]
   ...
   [-0.60815156 -1.9310684  -0.6283335  ...  2.0176606   0.6538963
    -3.2834167 ]
   [-0.63249457 -1.9342227  -0.6529601  ...  2.006361    0.6496838
    -3.2773361 ]
   [-0.

In [ ]:
bn_model = Model(inputs=model.input, outputs=model.get_layer('batch_normalization').output)
bn_output = bn_model.predict(x)
print(bn_output)

1/1 [==============================] - 0s 59ms/step
[[[[-7.67134070e-01 -8.60825658e-01 -8.07629764e-01 ...  9.01398733e-02
    -4.82812710e-02 -6.96215391e-01]
   [-8.38218093e-01 -8.65473390e-01 -8.14993858e-01 ...  1.18994743e-01
    -2.17981776e-03 -7.04599321e-01]
   [-8.03994358e-01 -8.73245895e-01 -7.78159738e-01 ...  1.25666797e-01
    -2.15805750e-02 -6.91847742e-01]
   ...
   [-9.46152985e-01 -1.10550046e+00 -9.14427817e-01 ...  3.34854335e-01
     2.36539841e-02 -8.92909825e-01]
   [-8.68387222e-01 -1.02869475e+00 -8.83404911e-01 ...  3.44448119e-01
     7.59588107e-02 -8.52572739e-01]
   [-9.48874772e-01 -1.09715164e+00 -9.21460509e-01 ...  3.38246047e-01
     2.11791173e-02 -8.80131662e-01]]

  [[-8.59614670e-01 -8.07327092e-01 -8.56887400e-01 ...  1.56431884e-01
     4.15038802e-02 -6.85148537e-01]
   [-7.22530127e-01 -9.01622474e-01 -8.67556214e-01 ...  1.64395541e-01
     4.54623476e-02 -6.87059581e-01]
   [-8.24730754e-01 -8.94706726e-01 -8.20680201e-01 ...  1.73389092

In [ ]:
a = model.get_layer('batch_normalization').get_weights()
weight = a[0]/pow(a[3]+0.001, 0.5)
bias = -(a[0]*a[2])/pow(a[3]+0.001, 0.5) + a[1]
print(weight)
print(bias)

[2.807677  1.9425796 2.18215   2.0206542 0.8150177 2.027068  2.5828292
 1.0363885]
[ 0.82028997  2.663373    0.51762754 -3.1442692   2.6822412  -3.7248764
 -1.695941    2.5184193 ]


In [ ]:
bn_output_1 = np.zeros((1,22,22,8))
for i in range(22):
  for j in range(22):
    for k in range(8):
      bn_output_1[0][i][j][k] = pool_output[0][i][j][k]*weight[k]+bias[k]
bn_output_1-bn_output

array([[[[ 1.19209290e-07,  1.19209290e-07,  0.00000000e+00, ...,
           2.30967999e-07,  7.82310963e-08, -2.38418579e-07],
         [ 1.19209290e-07,  1.19209290e-07, -5.96046448e-08, ...,
           2.08616257e-07,  7.59027898e-08, -5.96046448e-08],
         [ 5.96046448e-08, -5.96046448e-08, -5.96046448e-08, ...,
           5.96046448e-08, -1.86264515e-09, -5.96046448e-08],
         ...,
         [ 5.96046448e-08,  0.00000000e+00,  0.00000000e+00, ...,
           2.68220901e-07,  0.00000000e+00, -1.78813934e-07],
         [ 1.19209290e-07,  1.19209290e-07,  0.00000000e+00, ...,
          -2.98023224e-08,  3.72529030e-08, -1.78813934e-07],
         [ 5.96046448e-08,  1.19209290e-07, -5.96046448e-08, ...,
           2.98023224e-07,  8.19563866e-08, -5.96046448e-08]],

        [[ 5.96046448e-08,  5.96046448e-08,  0.00000000e+00, ...,
           2.98023224e-08,  2.60770321e-08, -1.78813934e-07],
         [ 1.19209290e-07,  1.78813934e-07, -5.96046448e-08, ...,
           2.98023224e